<a href="https://colab.research.google.com/github/AlexKressner/WS25_Supply_Chain_Optimierung/blob/main/Fallstudie_Juicy_AG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie JUICY AG (Network Design)

In [136]:
! pip install -q pyscipopt

In [137]:

import pandas as pd
from pyscipopt import Model, quicksum

## Repo & Daten laden, Bsp. Produktions- und Transportkosten

In [138]:
! git clone https://github.com/AlexKressner/WS25_Supply_Chain_Optimierung

fatal: destination path 'WS25_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [139]:
! ls

sample_data  WS25_Supply_Chain_Optimierung


In [140]:
! ls WS25_Supply_Chain_Optimierung/Daten/NetworkDesign/

NetworkDesign_Beschaffungskosten.csv  NetworkDesign_Nachfrage.csv
NetworkDesign_Invest_Kapa.csv	      NetworkDesign_Produktionskosten.csv


In [141]:
folder = "WS25_Supply_Chain_Optimierung/Daten/NetworkDesign"

In [142]:
operative_kosten = pd.read_csv(f"{folder}/NetworkDesign_Produktionskosten.csv", sep=";")

In [143]:
operative_kosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [144]:
operative_kosten.head()

,Produktionsstandort,Markt,Produktionskosten
0,Charleston,USA,81
1,Charleston,Südamerika,92
2,Charleston,Europa,101
3,Charleston,Asien,130
4,Charleston,Afrika,115


In [145]:
kapazitaeten = pd.read_csv(f"{folder}/NetworkDesign_Invest_Kapa.csv", sep=";")

In [146]:
kapazitaeten.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [147]:
nachfrage = pd.read_csv(f"{folder}/NetworkDesign_Nachfrage.csv", sep=";")

In [148]:
nachfrage

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


## Indexmengen

In [149]:
I = kapazitaeten.Produktionsstandort.unique()
J = list(nachfrage.Markt.unique())
A = kapazitaeten.Ausbaustufe.unique()

## Erweiterung Aufgabe 2

In [150]:
J.append("Arabische_Halbinsel")

In [151]:
J

['USA', 'Südamerika', 'Europa', 'Asien', 'Afrika', 'Arabische_Halbinsel']

## Parameter

In [152]:
# Produktions- und Transportkosten
operative_kosten.set_index(["Produktionsstandort","Markt"], inplace=True)
cp = operative_kosten.to_dict("dict")["Produktionskosten"]

## Erweiterung Aufgabe 2

In [153]:
cp["Charleston","Arabische_Halbinsel"]=125
cp["Curitiba","Arabische_Halbinsel"]=100
cp["Hamburg","Arabische_Halbinsel"]=100
cp["Quanzhou","Arabische_Halbinsel"]=75
cp["East London","Arabische_Halbinsel"]=85

In [154]:
# Fixkosten
cf = kapazitaeten.set_index(["Produktionsstandort","Ausbaustufe"])
cf = cf.to_dict("dict")["Investitionen"]

## Erweiterung Aufgabe 1)

In [155]:
cf['Charleston', 'klein'] = cf['Charleston', 'klein'] * 0.7
cf['Charleston', 'groß'] = cf['Charleston', 'groß'] * 0.7

In [156]:
cf

{('Charleston', 'klein'): 4200.0,
 ('Charleston', 'groß'): 6300.0,
 ('Curitiba', 'klein'): 4500,
 ('Curitiba', 'groß'): 6750,
 ('Hamburg', 'klein'): 6500,
 ('Hamburg', 'groß'): 9750,
 ('Quanzhou', 'klein'): 4100,
 ('Quanzhou', 'groß'): 6150,
 ('East London', 'klein'): 4000,
 ('East London', 'groß'): 6000}

In [157]:
# Kapazitäten
cap = kapazitaeten.set_index("Ausbaustufe")
cap = cap.to_dict("dict")["Kapazitäten"]
cap

{'klein': 10, 'groß': 20}

In [158]:
# Nachfrage
nachfrage.set_index("Markt", inplace=True)
d = nachfrage.to_dict("dict")["Nachfragemenge"]
d


{'USA': 12, 'Südamerika': 8, 'Europa': 14, 'Asien': 16, 'Afrika': 7}

## Erweiterung Aufgabe 1)

In [159]:
d["Arabische_Halbinsel"]=5

## Modellinstanz ziehen

In [160]:
model = Model()

## Entscheidungsvariablen

In [161]:
Y = {}
for i in I:
  for a in A:
    Y[i,a] = model.addVar(vtype="B", name=f"Ausbaustufe_{a}_Standort_{i}")

In [162]:
X = {}
for i in I:
  for j in J:
    X[i,j] = model.addVar(vtype="I", name=f"Produktion_Standort_{i}_für_Markt_{j}")

In [163]:
len(model.getVars())

40

## Zielfunktion

In [164]:
model.setObjective(
    quicksum(cp[i,j]*X[i,j] for i in I for j in J)
    + quicksum(cf[i,a]*Y[i,a] for i in I for a in A),
    sense="minimize"
    )

## Nebenbedingungen

In [165]:
# Nachfrage (5)
for j in J:
  model.addCons(quicksum(X[i,j] for i in I) >= d[j], name=f"Nachfrage_Markt_{j}")

In [166]:
# Kopplung Produktion und Investment (5)
for i in I:
  model.addCons(
      quicksum(X[i,j] for j in J) <= quicksum(cap[a]*Y[i,a] for a in A),
      name=f"Kapazität_Standort_{i}"
      )

In [167]:
# Nur eine Ausbaustufe (5)
for i in I:
  model.addCons(
      quicksum(Y[i,a] for a in A) <= 1,
      name=f"Ausbaustufe_{i}"
      )

In [168]:
# Mindestbeschäftigung
model.addCons(quicksum(X["Charleston",j] for j in J) >= 5, name="Mindestmenge_Charleston")
model.addCons(quicksum(X["Hamburg",j] for j in J) >= 20, name="Mindestmenge_Hamburg")

Mindestmenge_Hamburg

In [169]:
len(model.getConss())

18

## Lösung

In [170]:
model.optimize()

In [171]:
print('LÖSUNG:')
print('Zielfunktionswert (Kosten) =', model.getObjVal())
print("\n")
print("PRODUKTIONS-MARKT-ZUTEILUNG")
for j in J:
  print(f"{j}:")
  for i in I:
    if model.getVal(X[i,j])>0:
      print('\t', f'{i,j} =', round(model.getVal(X[i,j])))
print("\n")
print("INVESTITIONEN")
for i in I:
  for a in A:
    if model.getVal(Y[i,a])>0:
      print(f'{i,a} =', round(model.getVal(Y[i,a])))

LÖSUNG:
Zielfunktionswert (Kosten) = 31090.0


PRODUKTIONS-MARKT-ZUTEILUNG
USA:
	 ('Charleston', 'USA') = 10
	 ('Hamburg', 'USA') = 2
Südamerika:
	 ('Hamburg', 'Südamerika') = 4
	 ('East London', 'Südamerika') = 4
Europa:
	 ('Hamburg', 'Europa') = 14
Asien:
	 ('Quanzhou', 'Asien') = 16
Afrika:
	 ('East London', 'Afrika') = 7
Arabische_Halbinsel:
	 ('Quanzhou', 'Arabische_Halbinsel') = 4
	 ('East London', 'Arabische_Halbinsel') = 1


INVESTITIONEN
('Charleston', 'klein') = 1
('Hamburg', 'groß') = 1
('Quanzhou', 'groß') = 1
('East London', 'groß') = 1


In [172]:
model.getStatus()

'optimal'

LÖSUNG:
Zielfunktionswert (Kosten) = 23751.0


PRODUKTIONS-MARKT-ZUTEILUNG
USA:
	 ('Curitiba', 'USA') = 12
Südamerika:
	 ('Curitiba', 'Südamerika') = 8
Europa:
	 ('Quanzhou', 'Europa') = 4
	 ('East London', 'Europa') = 10
Asien:
	 ('Quanzhou', 'Asien') = 16
Afrika:
	 ('East London', 'Afrika') = 7


INVESTITIONEN
('Curitiba', 'groß') = 1
('Quanzhou', 'groß') = 1
('East London', 'groß') = 1